_Note: This is a SQL scoped notebook, and therefore should be attached to a Serverless SQL warehouse for execution._  

In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'fhir_workshop';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
SET VARIABLE catalog_use = :catalog_use; 
SET VARIABLE schema_use = :schema_use; 

In [0]:
USE IDENTIFIER(catalog_use || "." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

In [0]:
-- DROP TABLE IF EXISTS resources_exploded;

In [0]:
CREATE OR REFRESH STREAMING TABLE resources_exploded 
COMMENT 'Exploded FHIR Resources'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  bundle_uuid
  ,CAST(entry.value:fullUrl AS STRING) as fullUrl
  ,CAST(entry.value:resource.resourceType AS STRING) as resourceType
  ,resource.key as key
  ,resource.value as value
FROM 
  STREAM(fhir_bronze_variant)
  ,LATERAL variant_explode(fhir:entry) as entry
  ,LATERAL variant_explode(entry.value:resource) as resource

In [0]:
-- DROP TABLE IF EXISTS resource_schemas;

In [0]:
CREATE OR REFRESH STREAMING TABLE resource_schemas 
COMMENT 'Schemas of inferred from FHIR Resource Variant Data Types'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  resourceType
  ,key as resource
  ,schema_of_variant_agg(value) as schema_of_variant
  ,REPLACE(schema_of_variant_agg(value), 'OBJECT', 'STRUCT') as schema_as_struct
FROM STREAM(resources_exploded)
GROUP BY 
  resourceType
  ,key

In [0]:
select * from resource_schemas ORDER BY 1, 2 LIMIT 100;